In [ ]:
!pip install transformers

In [ ]:
!pip install ml_things

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.1 MB/s eta 0:00:00
  Created wheel for ml_things: filename=ml_things-0.0.1-py3-none-any.whl size=24393 sha256=84f3f21999f50dd2f139ac59cbf837dbe067785c35fa8890c164df7a0c6ebc53
  Stored in directory: /root/.cache/pip/wheels/2a/b9/36/3725744c1d8ebec00cd847c9461b4352ce08de9d3f20b6bdcf
Successfully built ml_things


In [ ]:
import io
import os
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW,
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)


In [ ]:
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df = pd.read_csv("data_scratch.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,TEXT,CREDIBILITY,SEVERITY,LABEL,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NVIDIA’S AI CHIP DOMINANCE IS BEING TARGETED B...,0,2,4,NaN,NaN,NaN,NaN,NaN,NaN
1,TO BUILD A BETTER AI SUPERCOMPUTER LET THERE B...,1,2,3,NaN,NaN,NaN,NaN,NaN,NaN
2,NVIDIA'S SHIELD TV PRO IS STILL THE BEST STREA...,1,2,3,NaN,NaN,NaN,NaN,NaN,NaN
3,ZUCK JUST REVEALED THE SECRET INGREDIENT IN HI...,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN
4,Asus ROG Zephyrus G14 Review: A Gaming Laptop ...,1,2,2,NaN,NaN,NaN,NaN,NaN,NaN
5,I'm a software engineer for Nvidia. Here's how...,1,3,3,NaN,NaN,NaN,NaN,NaN,NaN
6,Nvidia employees are getting richer with a 'sp...,1,3,4,NaN,NaN,NaN,NaN,NaN,NaN
7,Nvidia is about to face its first major threat...,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN
8,The world's biggest chipmaker just got hit by ...,3,8,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
X = df['TEXT']
X

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    NVIDIA’S AI CHIP DOMINANCE IS BEING TARGETED B...
1    TO BUILD A BETTER AI SUPERCOMPUTER LET THERE B...
2    NVIDIA'S SHIELD TV PRO IS STILL THE BEST STREA...
3    ZUCK JUST REVEALED THE SECRET INGREDIENT IN HI...
4    Asus ROG Zephyrus G14 Review: A Gaming Laptop ...
5    I'm a software engineer for Nvidia. Here's how...
6    Nvidia employees are getting richer with a 'sp...
7    Nvidia is about to face its first major threat...
8    The world's biggest chipmaker just got hit by ...
Name: TEXT, dtype: object

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class NewsDataset(Dataset):
  r"""PyTorch Dataset class for loading data.

  This is where the data parsing happens.

  This class is built with reusability in mind: it can be used as is as.

  Arguments:

    path (:obj:`str`):
        Path to the data partition.

  """

  def __init__(self, tokenizer, max_length=768):
    super(NewsDataset).__init__()
    self.texts = df['TEXT']
    self.labels=df['LABEL']
    self.credibilities = df['CREDIBILITY']
    self.severities = df['SEVERITY']

    # Number of exmaples.
    self.n_examples = len(self.labels)

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for i, txt in enumerate(self.texts):
      text = f'''
      Your task is to predict the sentiment of stock price of Nvidia given these news and severity of these news.
      A output of 0 means stock price likely will decrease by more than 10% tomorrow.
      A output of 1 means stock price likely will decrease by more than 5% but less than 10% tomorrow.
      A output of 2 means stock price likely will change by less than 2% tomorrow.
      A output of 3 means stock price likely will increase by more than 5% but less than 10% tomorrow tomorrow.
      A output of 4 means stock price likely will increase by more than 10% tomorrow.
      The severity score of the news is {self.severities[i]}.
      The credibility of the news is {self.credibilities[i]}.
      Content of the news: {txt}
      '''

      encodings_dict = tokenizer('<|startoftext|>'+ text + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]#, self.credibilities[idx], self.severities[idx]



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer
from torch import nn

class NewsClassifierModel(nn.Module):
    def __init__(self, config, num_additional_features=2, num_classes=5):
        super(NewsClassifierModel, self).__init__()
        # self.gpt_model = GPT2Model(config)
        self.gpt_model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path='gpt2', config=config)
        self.config = config

        # self.additional_features_fc = nn.Linear(num_additional_features, self.gpt_model.config.n_embd)
        self.gpt_model.resize_token_embeddings(len(tokenizer))
        self.gpt_model.config.pad_token_id = self.gpt_model.config.eos_token_id
        self.gpt_model.to(device)
        # self.dropout = nn.Dropout(gpt_model.config.dropout_rate)
        # self.classifier = nn.Linear(self.gpt_model.config.n_embd, num_classes)
        # self.softmax = nn.Softmax()

    def forward(self, b_input_ids, b_labels, b_masks):
        # Extract text embeddings from the GPT model
        # batch_size, sequence_length = b_input_ids.shape[:2]
        # if self.config.pad_token_id is None:
        #     sequence_lengths = -1
        # else:
        #     if b_input_ids is not None:
        #         sequence_lengths = torch.ne(b_input_ids, self.config.pad_token_id).sum(-1) - 1
        #     else:
        #         sequence_lengths = -1
        # text_outputs = self.gpt_model(  b_input_ids,
        #                   attention_mask = b_masks,
        #                   token_type_ids=None
        #                 )
        # print(sequence_lengths)
        # text_outputs = text_outputs[0]

        # # Process additional integer values
        # concat_severities_creds = concat_severities_creds.float()
        # feature_output = self.additional_features_fc(concat_severities_creds)

        # # Concatenate or add the additional features to the text embeddings
        # combined_embeddings = text_outputs + feature_output.unsqueeze(1)

        # # Apply dropout for regularization
        # # Pass the combined embeddings through the classifier
        # logits = self.classifier(combined_embeddings)
        # pooled_logits = logits[range(batch_size), sequence_lengths]
        # pooled_logits = self.softmax(pooled_logits)
        # print('pooled_logits', pooled_logits.shape)
        out = self.gpt_model(  b_input_ids,
                          attention_mask = b_masks,
                            labels=b_labels,
                          token_type_ids=None

                        )
        loss, logits = out[:2]

        return loss, logits

# Example usage
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# gpt_model = GPT2Model.from_pretrained('gpt2')

# Define the number of additional features and classes
num_additional_features = 2  # Adjust as needed
num_classes = 5  # Number of classes for classification

# Initialize custom classifier model


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path='gpt2')
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token

train_dataset = NewsDataset(tokenizer=tokenizer)

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            batch_size = 1 # Trains with this batch size.
        )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print('Loading configuraiton...')
model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path='gpt2', num_labels=5)

# Get model's tokenizer.
print('Loading tokenizer...')
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path='gpt2')
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token


# Get the actual model.
print('Loading model...')
model = NewsClassifierModel(model_config)

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Loading configuraiton...
Loading tokenizer...
Loading model...


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded to `cpu`


In [ ]:
epochs = 4

# Number of batches - depending on the max sequence length and GPU memory.
# For 512 sequence length batch of 10 works without cuda memory issues.
# For small sequence length can try batch of 32 or higher.
batch_size = 1

# Pad or truncate text sequences to a specific length
# if `None` it will use maximum sequence of word piece tokens allowed by model.
max_length = 60

# Look for gpu to use. Will use `cpu` by default if no gpu found.
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Name of transformers model - will use already pretrained model.
# Path of transformer model - will load your own model from local disk.
model_name_or_path = 'gpt2'

# Dictionary of labels and their id - this will be used to convert.
# String labels to number ids.
# labels_ids = {'neg': 0, 'pos': 1}

# How many labels are we using in training.
# This is used to decide size of classification head.
n_labels = 5
epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    criterion = nn.CrossEntropyLoss()
    for step, batch in enumerate(train_dataloader):
        # print(batch)
        b_input_ids = batch[0].to(device)
        b_attn_masks = batch[1].to(device)
        b_labels = batch[2].to(device)
        # b_credibilities= batch[3].to(device)
        # b_severities= batch[4].to(device)
        # b_cred_severities = torch.cat((b_credibilities, b_severities))

        model.zero_grad()

        loss, logits = model(  b_input_ids,
                          b_labels,
                          b_attn_masks
                          # b_cred_severities
                        )
        loss.retain_grad()
        print('loss', loss)
        print('logits ', logits)


        # loss = criterion(logits, b_labels)
        print('loss.grad', loss.grad)
        loss.backward()
        print('loss.grad', loss.grad)
        optimizer.step()
        print('loss.grad', loss.grad)
        scheduler.step()
        print('model.gpt_model.score.weight.grad ', model.gpt_model.score.weight.grad)

    #     loss = outputs[0]

    #     batch_loss = loss.item()
    #     total_train_loss += batch_loss

    #     # Get sample every x batches.
    #     if step % sample_every == 0 and not step == 0:

    #         elapsed = format_time(time.time() - t0)
    #         print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

    #         model.eval()

    #         sample_outputs = model.generate(
    #                                 bos_token_id=random.randint(1,30000),
    #                                 do_sample=True,
    #                                 top_k=50,
    #                                 max_length = 200,
    #                                 top_p=0.95,
    #                                 num_return_sequences=1
    #                             )
    #         for i, sample_output in enumerate(sample_outputs):
    #               print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

    #         model.train()

    #     loss.backward()

    #     optimizer.step()

    #     scheduler.step()

    # # Calculate the average loss over all of the batches.
    # avg_train_loss = total_train_loss / len(train_dataloader)

    # # Measure how long this epoch took.
    # training_time = format_time(time.time() - t0)

    # print("")
    # print("  Average training loss: {0:.2f}".format(avg_train_loss))
    # print("  Training epoch took: {:}".format(training_time))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



======== Epoch 1 / 5 ========
Training...
loss tensor(3.0002, grad_fn=<NllLossBackward0>)
logits  tensor([[ 0.0395, -0.2176,  0.7741,  1.0900, -1.0050]],
       grad_fn=<IndexBackward0>)
loss.grad None
loss.grad tensor(1.)
loss.grad tensor(1.)
model.gpt_model.score.weight.grad  tensor([[-0.0945,  0.1845,  0.0022,  ..., -0.0139, -0.0370,  0.0182],
        [-0.0730,  0.1427,  0.0017,  ..., -0.0108, -0.0286,  0.0141],
        [-0.1969,  0.3846,  0.0045,  ..., -0.0291, -0.0772,  0.0380],
        [-0.2701,  0.5274,  0.0062,  ..., -0.0399, -0.1059,  0.0521],
        [ 0.6344, -1.2391, -0.0145,  ...,  0.0937,  0.2488, -0.1225]])
loss tensor(0.6051, grad_fn=<NllLossBackward0>)
logits  tensor([[ 0.4853, -0.9279,  1.2947,  2.0402, -0.3201]],
       grad_fn=<IndexBackward0>)
loss.grad None
loss.grad tensor(1.)
loss.grad tensor(1.)
model.gpt_model.score.weight.grad  tensor([[-6.4538e-02,  1.3971e-01,  6.1578e-02,  ...,  5.1882e-04,
          8.9323e-04,  4.6754e-02],
        [-1.5706e-02,  3.3998